In [1]:
import pandas as pd 
import numpy as np

In [2]:
import os

In [3]:
from sklearn.model_selection import train_test_split

In [4]:
from sklearn.model_selection import RandomizedSearchCV

In [5]:
df = pd.read_csv("train.csv")

In [6]:
df.head()

,pet_id,issue_date,listing_date,condition,color_type,length(m),height(cm),X1,X2,breed_category,pet_category
0,ANSL_69903,2016-07-10 00:00:00,2016-09-21 16:25:00,2.0,Brown Tabby,0.80,7.78,13,9,0.0,1
1,ANSL_66892,2013-11-21 00:00:00,2018-12-27 17:47:00,1.0,White,0.72,14.19,13,9,0.0,2
2,ANSL_69750,2014-09-28 00:00:00,2016-10-19 08:24:00,NaN,Brown,0.15,40.90,15,4,2.0,4
3,ANSL_71623,2016-12-31 00:00:00,2019-01-25 18:30:00,1.0,White,0.62,17.82,0,1,0.0,2
4,ANSL_57969,2017-09-28 00:00:00,2017-11-19 09:38:00,2.0,Black,0.50,11.06,18,4,0.0,1


In [7]:
df1 = pd.read_csv("test.csv")
df1.head()

,pet_id,issue_date,listing_date,condition,color_type,length(m),height(cm),X1,X2
0,ANSL_75005,17-08-2005 00:00,07-09-2017 15:35,0.0,Black,0.87,42.73,0,7
1,ANSL_76663,15-11-2018 00:00,08-05-2019 17:24,1.0,Orange Tabby,0.06,6.71,0,1
2,ANSL_58259,11-10-2012 00:00,02-04-2018 16:51,1.0,Black,0.24,41.21,0,7
3,ANSL_67171,13-02-2015 00:00,06-04-2018 07:25,1.0,Black,0.29,8.46,7,1
4,ANSL_72871,18-01-2017 00:00,26-04-2018 13:42,1.0,Brown,0.71,30.92,0,7


In [9]:
# Missing values treatment #

df['condition'] = df['condition'].fillna(3)
df.isnull().sum()

pet_id            0
issue_date        0
listing_date      0
condition         0
color_type        0
length(m)         0
height(cm)        0
X1                0
X2                0
breed_category    0
pet_category      0
dtype: int64

In [10]:
df["listing_date"] = (df["listing_date"].str.split(" ", n = 1, expand = True))[0] 
df["listing_date"].head()

0    2016-09-21
1    2018-12-27
2    2016-10-19
3    2019-01-25
4    2017-11-19
Name: listing_date, dtype: object

In [11]:
df["issue_date"] = (df["issue_date"].str.split(" ", n = 1, expand = True))[0] 
df["issue_date"].head()

0    2016-07-10
1    2013-11-21
2    2014-09-28
3    2016-12-31
4    2017-09-28
Name: issue_date, dtype: object

In [12]:
df['listing_date'] =pd.to_datetime(df['listing_date'], format='%Y-%m-%d')
df['listing_date'].head()

0   2016-09-21
1   2018-12-27
2   2016-10-19
3   2019-01-25
4   2017-11-19
Name: listing_date, dtype: datetime64[ns]

In [13]:
df['issue_date'] =pd.to_datetime(df['issue_date'], format='%Y-%m-%d')
df['issue_date'].head()

0   2016-07-10
1   2013-11-21
2   2014-09-28
3   2016-12-31
4   2017-09-28
Name: issue_date, dtype: datetime64[ns]

In [14]:
df['No_days'] = (df['listing_date']-df['issue_date']).dt.days  # This will convert no. of days into integer format $#
df['No_days'].head()

0      73
1    1862
2     752
3     755
4      52
Name: No_days, dtype: int64

In [15]:
# there is no requirement of issues date and listing date, as well the pet_id so will drop this columns #

df = df.drop(['pet_id','issue_date','listing_date'],axis=1)
df.head()

,condition,color_type,length(m),height(cm),X1,X2,breed_category,pet_category,No_days
0,2.0,Brown Tabby,0.80,7.78,13,9,0.0,1,73
1,1.0,White,0.72,14.19,13,9,0.0,2,1862
2,3.0,Brown,0.15,40.90,15,4,2.0,4,752
3,1.0,White,0.62,17.82,0,1,0.0,2,755
4,2.0,Black,0.50,11.06,18,4,0.0,1,52


In [ ]:
df.head()

,condition,color_type,length(m),height(cm),X1,X2,breed_category,pet_category,No_days
0,2.0,Brown Tabby,0.80,7.78,13,9,0.0,1,73
1,1.0,White,0.72,14.19,13,9,0.0,2,1862
2,3.0,Brown,0.15,40.90,15,4,2.0,4,752
3,1.0,White,0.62,17.82,0,1,0.0,2,755
4,2.0,Black,0.50,11.06,18,4,0.0,1,52


In [16]:
# Dropping the length and height as they do not have a significanr relationship with the yarget variables

df = df.drop(['height(cm)','length(m)'],axis=1)
df.head()


,condition,color_type,X1,X2,breed_category,pet_category,No_days
0,2.0,Brown Tabby,13,9,0.0,1,73
1,1.0,White,13,9,0.0,2,1862
2,3.0,Brown,15,4,2.0,4,752
3,1.0,White,0,1,0.0,2,755
4,2.0,Black,18,4,0.0,1,52


In [17]:
df.info()

#Handling the categorical variable colour #
from sklearn import preprocessing
le = preprocessing.LabelEncoder()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 18834 entries, 0 to 18833
Data columns (total 7 columns):
 #   Column          Non-Null Count  Dtype  
---  ------          --------------  -----  
 0   condition       18834 non-null  float64
 1   color_type      18834 non-null  object 
 2   X1              18834 non-null  int64  
 3   X2              18834 non-null  int64  
 4   breed_category  18834 non-null  float64
 5   pet_category    18834 non-null  int64  
 6   No_days         18834 non-null  int64  
dtypes: float64(2), int64(4), object(1)
memory usage: 1.0+ MB


In [18]:
x_cat = df['color_type']
x_cat.head()

0    Brown Tabby
1          White
2          Brown
3          White
4          Black
Name: color_type, dtype: object

In [19]:
X_cat = pd.DataFrame(le.fit_transform(x_cat))
X_cat.head()

,0
0,18
1,53
2,15
3,53
4,2


In [20]:
# We need ro concart this new colour series to our original data set #

X = pd.concat([df,X_cat],axis=1)
X.head()

,condition,color_type,X1,X2,breed_category,pet_category,No_days,0
0,2.0,Brown Tabby,13,9,0.0,1,73,18
1,1.0,White,13,9,0.0,2,1862,53
2,3.0,Brown,15,4,2.0,4,752,15
3,1.0,White,0,1,0.0,2,755,53
4,2.0,Black,18,4,0.0,1,52,2


In [21]:
X.drop(['color_type'],axis=1,inplace=True)
X.head()

,condition,X1,X2,breed_category,pet_category,No_days,0
0,2.0,13,9,0.0,1,73,18
1,1.0,13,9,0.0,2,1862,53
2,3.0,15,4,2.0,4,752,15
3,1.0,0,1,0.0,2,755,53
4,2.0,18,4,0.0,1,52,2


In [22]:
# Also we needd to drop the two target variables pet_catergory and breed_category #

X.rename(columns = {0:'color'}, inplace = True) 

X.head()

,condition,X1,X2,breed_category,pet_category,No_days,color
0,2.0,13,9,0.0,1,73,18
1,1.0,13,9,0.0,2,1862,53
2,3.0,15,4,2.0,4,752,15
3,1.0,0,1,0.0,2,755,53
4,2.0,18,4,0.0,1,52,2


In [23]:
# We will drop both the target variables from this # Independent cariables #

x = X.drop(['breed_category','pet_category'],axis=1)
x.head()

,condition,X1,X2,No_days,color
0,2.0,13,9,73,18
1,1.0,13,9,1862,53
2,3.0,15,4,752,15
3,1.0,0,1,755,53
4,2.0,18,4,52,2


In [24]:
# Identifying yhe categort variables, as they need to be mentioned separately for applying the catboosting algorithm #

cat_features_index = [0,4]   # Condition & Color are categorical variables 

In [25]:
x['condition'] = x['condition'].astype('int64')
x.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 18834 entries, 0 to 18833
Data columns (total 5 columns):
 #   Column     Non-Null Count  Dtype
---  ------     --------------  -----
 0   condition  18834 non-null  int64
 1   X1         18834 non-null  int64
 2   X2         18834 non-null  int64
 3   No_days    18834 non-null  int64
 4   color      18834 non-null  int64
dtypes: int64(5)
memory usage: 735.8 KB


In [26]:
x.head()

,condition,X1,X2,No_days,color
0,2,13,9,73,18
1,1,13,9,1862,53
2,3,15,4,752,15
3,1,0,1,755,53
4,2,18,4,52,2


In [27]:
# Target variables : Breed_categpru & Pet categpory 
# Differentating yhe independnetn & dependent variables #
y = X['breed_category']  # 1. target variable : Breed_category
y.head()

0    0.0
1    0.0
2    2.0
3    0.0
4    0.0
Name: breed_category, dtype: float64

In [28]:
y1 = X['pet_category']  # 2. target variable : Pet_category
y1.head()

0    1
1    2
2    4
3    2
4    1
Name: pet_category, dtype: int64

In [29]:
# Applying the same above prepreccing steps for the test data set # # test data set - df1

# Removing the time portion for issue data and listing dates 

df1["listing_date"] = (df1["listing_date"].str.split(" ", n = 1, expand = True))[0] 
df1["listing_date"].head()

0    07-09-2017
1    08-05-2019
2    02-04-2018
3    06-04-2018
4    26-04-2018
Name: listing_date, dtype: object

In [30]:
df1["issue_date"] = (df1["issue_date"].str.split(" ", n = 1, expand = True))[0] 
df1["issue_date"].head()

0    17-08-2005
1    15-11-2018
2    11-10-2012
3    13-02-2015
4    18-01-2017
Name: issue_date, dtype: object

In [31]:
df1['listing_date'] =pd.to_datetime(df1['listing_date'], format='%d-%m-%Y')
df1['listing_date'].head()

0   2017-09-07
1   2019-05-08
2   2018-04-02
3   2018-04-06
4   2018-04-26
Name: listing_date, dtype: datetime64[ns]

In [32]:
df1['issue_date'] =pd.to_datetime(df1['issue_date'], format='%d-%m-%Y')
df1['issue_date'].head()

0   2005-08-17
1   2018-11-15
2   2012-10-11
3   2015-02-13
4   2017-01-18
Name: issue_date, dtype: datetime64[ns]

In [33]:
# Identifuy the no. of days between listuing date and issue date #

df1['No_days'] = (df1['listing_date']-df1['issue_date']).dt.days
df1['No_days'].head()

0    4404
1     174
2    1999
3    1148
4     463
Name: No_days, dtype: int64

In [34]:
df1.head()

,pet_id,issue_date,listing_date,condition,color_type,length(m),height(cm),X1,X2,No_days
0,ANSL_75005,2005-08-17,2017-09-07,0.0,Black,0.87,42.73,0,7,4404
1,ANSL_76663,2018-11-15,2019-05-08,1.0,Orange Tabby,0.06,6.71,0,1,174
2,ANSL_58259,2012-10-11,2018-04-02,1.0,Black,0.24,41.21,0,7,1999
3,ANSL_67171,2015-02-13,2018-04-06,1.0,Black,0.29,8.46,7,1,1148
4,ANSL_72871,2017-01-18,2018-04-26,1.0,Brown,0.71,30.92,0,7,463


In [35]:
# Replacing the null values in the condition with a value of 3 

df1['condition'] = df1['condition'].fillna(3)
df1.isnull().sum()

pet_id          0
issue_date      0
listing_date    0
condition       0
color_type      0
length(m)       0
height(cm)      0
X1              0
X2              0
No_days         0
dtype: int64

In [36]:
df2 = df1.drop(['pet_id','issue_date','listing_date','length(m)','height(cm)'],axis=1)


In [37]:
df2.head()

,condition,color_type,X1,X2,No_days
0,0.0,Black,0,7,4404
1,1.0,Orange Tabby,0,1,174
2,1.0,Black,0,7,1999
3,1.0,Black,7,1,1148
4,1.0,Brown,0,7,463


In [38]:
x_cat1 = df2['color_type']
x_cat1.head()

0           Black
1    Orange Tabby
2           Black
3           Black
4           Brown
Name: color_type, dtype: object

In [39]:
# Converting the color_type into labels with help of label encoder #


cat = pd.Series(le.transform(x_cat1))

In [40]:
X1 = pd.concat([df2,cat],axis=1)

In [ ]:
X1.head()

,condition,color_type,X1,X2,No_days,0
0,0.0,Black,0,7,4404,2
1,1.0,Orange Tabby,0,1,174,38
2,1.0,Black,0,7,1999,2
3,1.0,Black,7,1,1148,2
4,1.0,Brown,0,7,463,15


In [41]:
X1 = X1.drop(['color_type'],axis=1)
X1.head()

,condition,X1,X2,No_days,0
0,0.0,0,7,4404,2
1,1.0,0,1,174,38
2,1.0,0,7,1999,2
3,1.0,7,1,1148,2
4,1.0,0,7,463,15


In [42]:

X1.rename(columns = {0:'color'}, inplace = True) 

X1.head()

,condition,X1,X2,No_days,color
0,0.0,0,7,4404,2
1,1.0,0,1,174,38
2,1.0,0,7,1999,2
3,1.0,7,1,1148,2
4,1.0,0,7,463,15


In [43]:
X1['condition'] = X1['condition'].astype('int64')
X1.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 8072 entries, 0 to 8071
Data columns (total 5 columns):
 #   Column     Non-Null Count  Dtype
---  ------     --------------  -----
 0   condition  8072 non-null   int64
 1   X1         8072 non-null   int64
 2   X2         8072 non-null   int64
 3   No_days    8072 non-null   int64
 4   color      8072 non-null   int64
dtypes: int64(5)
memory usage: 315.4 KB


In [ ]:
df1.head()

,pet_id,issue_date,listing_date,condition,color_type,length(m),height(cm),X1,X2,No_days
0,ANSL_75005,2005-08-17,2017-09-07,0.0,Black,0.87,42.73,0,7,4404
1,ANSL_76663,2018-11-15,2019-05-08,1.0,Orange Tabby,0.06,6.71,0,1,174
2,ANSL_58259,2012-10-11,2018-04-02,1.0,Black,0.24,41.21,0,7,1999
3,ANSL_67171,2015-02-13,2018-04-06,1.0,Black,0.29,8.46,7,1,1148
4,ANSL_72871,2017-01-18,2018-04-26,1.0,Brown,0.71,30.92,0,7,463


In [44]:
X1.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 8072 entries, 0 to 8071
Data columns (total 5 columns):
 #   Column     Non-Null Count  Dtype
---  ------     --------------  -----
 0   condition  8072 non-null   int64
 1   X1         8072 non-null   int64
 2   X2         8072 non-null   int64
 3   No_days    8072 non-null   int64
 4   color      8072 non-null   int64
dtypes: int64(5)
memory usage: 315.4 KB


In [45]:
# Feature scaling : converting into standard scaler form #

from sklearn.linear_model import LogisticRegression
from sklearn.svm import SVC

In [46]:
# Lets build the logistic regression model #

lm = LogisticRegression()
mo = lm.fit(x,y)


/usr/local/lib/python3.6/dist-packages/sklearn/linear_model/_logistic.py:940: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  extra_warning_msg=_LOGISTIC_SOLVER_CONVERGENCE_MSG)


In [47]:

from sklearn.metrics import accuracy_score
from sklearn.metrics import confusion_matrix

# classification report and confusion matrix #
y_pred = mo.predict(x)

con = confusion_matrix(y_pred,y)
print(con)


acc = accuracy_score(y_pred,y)
print(acc)

[[6931 2946  836]
 [1522 5067    0]
 [ 547  344  641]]
0.671073590315387


In [48]:
# Random Search Algorithm #
params_gr = {'n_estimators':[103,104,105,106],'min_samples_leaf':[18,19,20],'max_depth':[9,10,11],'criterion':['entropy'],'max_leaf_nodes':[16,17,18,19],'max_features':[4]}

In [49]:
from sklearn.ensemble import RandomForestClassifier

In [128]:
class_weight = dict({0:1,1:2,1:100})

In [117]:
r_wcl= RandomForestClassifier(criterion='entropy', max_depth=9, max_features=4, max_leaf_nodes=19, min_samples_leaf=18)

In [118]:
score_wo = cross_val_score(r_wcl,x,y,cv=3)
score_wo.mean()

0.9066581713921632

In [90]:
from sklearn.model_selection import cross_val_score

In [129]:
r_cl= RandomForestClassifier(class_weight=class_weight,n_estimators=103,criterion='entropy', max_depth=9, max_features=4, max_leaf_nodes=19, min_samples_leaf=18)

In [130]:
score_wi = cross_val_score(r_cl,x,y,cv=3)
score_wi.mean()

0.8841987894233833

In [ ]:
# Adding the weights, as the problem seems to little unbalanced #
# Classes Present for Breed category : (0 : 9000 , 1: 8357 , 2: 1477), giving importance to class 2

r= RandomForestClassifier()

In [50]:
from sklearn.model_selection import RandomizedSearchCV
from sklearn.model_selection import GridSearchCV


In [51]:
r= RandomForestClassifier()

In [52]:
grid = GridSearchCV(r,params_gr,cv=7)
mo_rf_gr = grid.fit(x,y)

In [53]:
print(mo_rf_gr.best_estimator_)

print(mo_rf_gr.best_params_)

print(mo_rf_gr.best_score_)

RandomForestClassifier(bootstrap=True, ccp_alpha=0.0, class_weight=None,
                       criterion='entropy', max_depth=9, max_features=4,
                       max_leaf_nodes=19, max_samples=None,
                       min_impurity_decrease=0.0, min_impurity_split=None,
                       min_samples_leaf=18, min_samples_split=2,
                       min_weight_fraction_leaf=0.0, n_estimators=105,
                       n_jobs=None, oob_score=False, random_state=None,
                       verbose=0, warm_start=False)
{'criterion': 'entropy', 'max_depth': 9, 'max_features': 4, 'max_leaf_nodes': 19, 'min_samples_leaf': 18, 'n_estimators': 105}
0.9071356242844848


In [ ]:
#{'criterion': 'entropy', 'max_depth': 9, 'max_features': 4, 'max_leaf_nodes': 19, 'min_samples_leaf': 18, 'n_estimators': 105}
# Model for breed category (name :breed_rf1.pkl ) , Score : 0.9071356242844848

In [55]:
import pickle

In [56]:
# Saving this random forest model for breed category #

pickle.dump(mo_rf_gr,open('breed_rf1.pkl','wb'))

In [78]:
p = pickle.load(open('breed_rf.pkl','rb'))

In [79]:
# Identifying he values of y_test for model#

y_test_br = p.predict(X1)

In [80]:

y_test_br = pd.Series(y_test_br,name='breed_category')
y_test_br.head()

0    1.0
1    0.0
2    0.0
3    0.0
4    0.0
Name: breed_category, dtype: float64

In [ ]:
# applying the XG Boosting for the Pet Category #
! pip install xgboost

In [61]:
import xgboost

In [62]:
from xgboost import XGBClassifier

In [70]:
classifier = XGBClassifier()

In [64]:

params_xg={
 "learning_rate"    : [0.04,0.05, 0.10, 0.13,0.15, 0.17,0.20] ,
 "max_depth"        : [ 5,6,7,8,10],
 "min_child_weight" : [0.28,0.3,0.32,0.35,0.4,0.45],
 "gamma"            : [ 0.3,0.4,0.45,0.5,0.55],
 "colsample_bytree" : [ 0.5,0.6,0.65,0.7,0.75,0.8 ]
    
}

# learning rate : 0.05, colsample_bytree:0.7, min_child_weight:0.5

In [65]:
random_search=RandomizedSearchCV(classifier,param_distributions=params_xg,n_iter=100,n_jobs=-1,cv=7,verbose=3)

In [66]:
model1 = random_search.fit(x,y1)

Fitting 7 folds for each of 100 candidates, totalling 700 fits


[Parallel(n_jobs=-1)]: Using backend LokyBackend with 2 concurrent workers.
[Parallel(n_jobs=-1)]: Done  28 tasks      | elapsed:  1.0min
[Parallel(n_jobs=-1)]: Done 124 tasks      | elapsed:  3.9min
[Parallel(n_jobs=-1)]: Done 284 tasks      | elapsed:  9.6min
[Parallel(n_jobs=-1)]: Done 508 tasks      | elapsed: 17.2min
[Parallel(n_jobs=-1)]: Done 700 out of 700 | elapsed: 24.3min finished


In [67]:
print(model1.best_score_)
print(model1.best_params_)
print(model1.best_estimator_)

0.9061802554610053
{'min_child_weight': 0.4, 'max_depth': 8, 'learning_rate': 0.15, 'gamma': 0.45, 'colsample_bytree': 0.65}
XGBClassifier(base_score=0.5, booster='gbtree', colsample_bylevel=1,
              colsample_bynode=1, colsample_bytree=0.65, gamma=0.45,
              learning_rate=0.15, max_delta_step=0, max_depth=8,
              min_child_weight=0.4, missing=None, n_estimators=100, n_jobs=1,
              nthread=None, objective='multi:softprob', random_state=0,
              reg_alpha=0, reg_lambda=1, scale_pos_weight=1, seed=None,
              silent=None, subsample=1, verbosity=1)


In [ ]:
# Model : pet_xgboost : 90.54, pet_xgboost_n : 90.60, pet_xgboost_n1 : 90.61

In [ ]:
# Model : pet_xgboost_n1 : 90.61, cv=7
# Parameters : {'min_child_weight': 0.4, 'max_depth': 8, 'learning_rate': 0.15, 'gamma': 0.45, 'colsample_bytree': 0.65}

In [71]:
# Saving the model of Xgboosting # - Pet Categpry #

pickle.dump(model1,open('pet_xgboost_n1.pkl','wb'))

In [72]:
# loading the model #

q = pickle.load(open('pet_xgboost_n1.pkl','rb'))

In [73]:
y_pet_xg = q.predict(X1)

In [74]:

y_pet_xg= pd.Series(y_pet_xg,name='pet_category')
y_pet_xg.head()

0    2
1    1
2    2
3    2
4    2
Name: pet_category, dtype: int64

In [81]:
df_rf_xg= pd.concat([df1['pet_id'],y_test_br,y_pet_xg],axis=1) 
df_rf_xg.head()

,pet_id,breed_category,pet_category
0,ANSL_75005,1.0,2
1,ANSL_76663,0.0,1
2,ANSL_58259,0.0,2
3,ANSL_67171,0.0,2
4,ANSL_72871,0.0,2


In [82]:
# Saving the output in the CSV file # 
# Breed - breed_rf (90.75)
#Pet - pet_xgboost_n1 (90.61)
# Score :90.59
df_rf_xg.to_csv('submission_37.csv', index=False) 

In [ ]:
# Saving the output in the CSV file # 
# Breed - breed_rf1 (90.71)
#Pet - pet_xgboost_n1 (90.61)
# Score :90.63
df_rf_xg.to_csv('submission_36.csv', index=False) 

In [ ]:
# Saving the output in the CSV file #

df_rf_xg.to_csv('submission_35.csv', index=False) 

In [ ]:
df_rf_xg.to_csv('submission_31.csv', index=False)

In [ ]:
# Saving the output in the CSV file #

df_rf_xg.to_csv('submission_27.csv', index=False)   #Score :90.82 

In [ ]:
! pip install catboost

     |████████████████████████████████| 65.9MB 44kB/s 


In [ ]:
# Starting for Cat Boosting #
from catboost import Pool, CatBoostClassifier 

In [ ]:
model = CatBoostClassifier(iterations=10,
                           learning_rate=1,
                           depth=2,
                           loss_function='MultiClass')

In [ ]:
train_dataset = Pool(data=x,
                     label=y,
                     cat_features=cat_features_index)

In [ ]:
model.fit(train_dataset)

0:	learn: 0.3778418	total: 65ms	remaining: 585ms
1:	learn: 0.2557406	total: 81.4ms	remaining: 326ms
2:	learn: 0.1818474	total: 96.6ms	remaining: 225ms
3:	learn: 0.1648780	total: 112ms	remaining: 168ms
4:	learn: 0.1547720	total: 127ms	remaining: 127ms
5:	learn: 0.1529171	total: 141ms	remaining: 94.3ms
6:	learn: 0.1520109	total: 159ms	remaining: 68ms
7:	learn: 0.1500955	total: 173ms	remaining: 43.3ms
8:	learn: 0.1497249	total: 189ms	remaining: 20.9ms
9:	learn: 0.1481132	total: 206ms	remaining: 0us


In [ ]:
# Identifying he values of y_test for breed_category#

y_cat_br = model.predict(X1)

In [ ]:
y_cat_br

array([[1.],
       [0.],
       [0.],
       ...,
       [1.],
       [2.],
       [1.]])

In [ ]:
y_cat_br= pd.DataFrame(y_cat_br)
y_cat_br.head()

,0
0,1.0
1,0.0
2,0.0
3,0.0
4,0.0


In [ ]:

y_cat_br.rename(columns = {0:'breed_category'}, inplace = True) 

y_cat_br.head()

,breed_category
0,1.0
1,0.0
2,0.0
3,0.0
4,0.0


In [ ]:
train_dataset1 = Pool(data=x,
                     label=y1,
                     cat_features=cat_features_index)

In [ ]:
model1 = CatBoostClassifier(iterations=10,
                           learning_rate=1,
                           depth=2,
                           loss_function='MultiClass')

In [ ]:
model1.fit(train_dataset1)

0:	learn: 0.7637850	total: 21.2ms	remaining: 191ms
1:	learn: 0.4795030	total: 44.9ms	remaining: 180ms
2:	learn: 0.3371335	total: 62.2ms	remaining: 145ms
3:	learn: 0.3202560	total: 77.6ms	remaining: 116ms
4:	learn: 0.3049644	total: 93.6ms	remaining: 93.6ms
5:	learn: 0.2967946	total: 111ms	remaining: 73.8ms
6:	learn: 0.2939652	total: 133ms	remaining: 56.9ms
7:	learn: 0.2915995	total: 155ms	remaining: 38.9ms
8:	learn: 0.2856236	total: 173ms	remaining: 19.2ms
9:	learn: 0.2834072	total: 189ms	remaining: 0us


In [ ]:
# Identifying he values of y_test for pet_category#

y_cat_pet = model1.predict(X1)

In [ ]:
y_cat_pet= pd.DataFrame(y_cat_pet)
y_cat_pet.head()

,0
0,2
1,1
2,2
3,2
4,2


In [ ]:
y_cat_pet.rename(columns = {0:'pet_category'}, inplace = True) 

y_cat_pet.head()

,pet_category
0,2
1,1
2,2
3,2
4,2


In [ ]:
# Concatenating the output for breed and pet category #

df_cat= pd.concat([df1['pet_id'],y_cat_br['breed_category'],y_cat_pet['pet_category']],axis=1) 
df_cat.head()

,pet_id,breed_category,pet_category
0,ANSL_75005,1.0,2
1,ANSL_76663,0.0,1
2,ANSL_58259,0.0,2
3,ANSL_67171,0.0,2
4,ANSL_72871,0.0,2


In [ ]:
df_cat.to_csv('submission_28.csv', index=False)

In [ ]:
# Applying the Hyper parameter tuning in catboost Algorithm #

In [ ]:
from sklearn.model_selection import GridSearchCV

In [ ]:
dataset = Pool(data=x,
                     label=y,
                     cat_features=cat_features_index)

In [ ]:
cb1 = CatBoostClassifier(loss_function='MultiClass')

In [ ]:
params = {'depth': [7, 10],
          'learning_rate' : [0.03, 0.1],
         'l2_leaf_reg': [1,4]}

In [ ]:
cb_model = cb1.grid_search(params,dataset, cv=2, shuffle=True, search_by_train_test_split=True, verbose=2)




bestTest = 0.1489967316
bestIteration = 421

0:	loss: 0.1489967	best: 0.1489967 (0)	total: 35.2s	remaining: 4m 6s

bestTest = 0.1491655438
bestIteration = 131


bestTest = 0.1492250511
bestIteration = 574

2:	loss: 0.1492251	best: 0.1489967 (0)	total: 1m 47s	remaining: 2m 59s

bestTest = 0.1496772079
bestIteration = 151


bestTest = 0.1505391036
bestIteration = 291

4:	loss: 0.1505391	best: 0.1489967 (0)	total: 3m 48s	remaining: 2m 17s

bestTest = 0.1506815766
bestIteration = 106


bestTest = 0.1502450969
bestIteration = 369

6:	loss: 0.1502451	best: 0.1489967 (0)	total: 6m 40s	remaining: 57.3s

bestTest = 0.1508474343
bestIteration = 121

7:	loss: 0.1508474	best: 0.1489967 (0)	total: 8m 10s	remaining: 0us
Estimating final quality...


In [ ]:
y_test = cb1.predict_proba(X1)
y_test

array([[3.22440777e-05, 9.99950227e-01, 1.75288705e-05],
       [9.99976618e-01, 1.82206477e-05, 5.16099735e-06],
       [9.99985698e-01, 1.16169941e-05, 2.68486426e-06],
       ...,
       [1.46046855e-05, 9.99975885e-01, 9.51071277e-06],
       [5.50420951e-05, 3.59503532e-04, 9.99585454e-01],
       [1.10224179e-05, 9.99980978e-01, 7.99985046e-06]])

In [ ]:
y_test_br = cb1.predict(X1)
y_test_br

array([[1.],
       [0.],
       [0.],
       ...,
       [1.],
       [2.],
       [1.]])

In [ ]:
y_test_br= pd.DataFrame(y_test_br)
y_test_br.head()

,0
0,1.0
1,0.0
2,0.0
3,0.0
4,0.0


In [ ]:
y_test_br.rename(columns = {0:'breed_category'}, inplace = True) 

y_test_br.head()

,breed_category
0,1.0
1,0.0
2,0.0
3,0.0
4,0.0


In [ ]:
cb2 = CatBoostClassifier(loss_function='MultiClass')

In [ ]:
cb2.best_score_

{'learn': {'MultiClass': 0.15876372157008026}}

In [ ]:
cb2.classes_

array([0, 1, 2, 4])

In [ ]:
dataset1 = Pool(data=x,
                     label=y1,
                     cat_features=cat_features_index)

In [ ]:
cb_model1 = cb2.grid_search(params,dataset1, cv=2, shuffle=True, search_by_train_test_split=True, verbose=2)



bestTest = 0.2323637942
bestIteration = 967

0:	loss: 0.2323638	best: 0.2323638 (0)	total: 54.5s	remaining: 6m 21s

bestTest = 0.231189663
bestIteration = 278


bestTest = 0.2332272633
bestIteration = 999

2:	loss: 0.2332273	best: 0.2311897 (1)	total: 2m 44s	remaining: 4m 34s

bestTest = 0.2320703259
bestIteration = 449


bestTest = 0.2332052257
bestIteration = 510

4:	loss: 0.2332052	best: 0.2311897 (1)	total: 5m 52s	remaining: 3m 31s

bestTest = 0.2331446697
bestIteration = 136


bestTest = 0.2345197654
bestIteration = 706

6:	loss: 0.2345198	best: 0.2311897 (1)	total: 10m 22s	remaining: 1m 28s

bestTest = 0.2332180534
bestIteration = 273

7:	loss: 0.2332181	best: 0.2311897 (1)	total: 12m 41s	remaining: 0us
Estimating final quality...


In [ ]:
y_test_pet = cb2.predict(X1)
y_test_pet

array([[2],
       [1],
       [2],
       ...,
       [2],
       [4],
       [2]])

In [ ]:
y_test_pet = pd.DataFrame(y_test_pet)
y_test_pet.head()

,0
0,2
1,1
2,2
3,2
4,2


In [ ]:
y_test_pet.rename(columns={0:'pet_category'}, inplace=True)
y_test_pet.head()



,pet_category
0,2
1,1
2,2
3,2
4,2


In [ ]:
# Concatenating the output for breed and pet category #

df_cat_1= pd.concat([df1['pet_id'],y_test_br['breed_category'],y_test_pet['pet_category']],axis=1) 
df_cat_1.head()

,pet_id,breed_category,pet_category
0,ANSL_75005,1.0,2
1,ANSL_76663,0.0,1
2,ANSL_58259,0.0,2
3,ANSL_67171,0.0,2
4,ANSL_72871,0.0,2


In [ ]:
df_cat_1.to_csv('submission_29.csv', index=False)

In [ ]:
#Applning a new model with randomized search #

#Parameters for tuning
ra_grid = {'learning_rate': [0.03,0.25,0.5,0.1,1],
        'depth': [4,6,8,10],
        'l2_leaf_reg': [1, 3, 5, 7, 9]}

In [ ]:
randomized_search(grid,
                  X,
                  y=None,
                  cv=3,
                  n_iter=10,
                  partition_random_seed=0,
                  calc_cv_statistics=True, 
                  search_by_train_test_split=True,
                  refit=True, 
                  shuffle=True, 
                  stratified=None, 
                  train_size=0.8, 
                  verbose=True)

In [ ]:
c = CatBoostClassifier(loss_function='MultiClass')

In [ ]:
#Applyning a new model with randomized search #

randomized_breed = c.randomized_search(grid,
                  dataset,
                  cv=3,
                  n_iter=50,
                  partition_random_seed=0,
                  calc_cv_statistics=True, 
                  search_by_train_test_split=True,
                  refit=True, 
                  shuffle=True, 
                  stratified=None, 
                  verbose=True)


bestTest = 0.1494285245
bestIteration = 18

0:	loss: 0.1494285	best: 0.1494285 (0)	total: 21.1s	remaining: 17m 12s

bestTest = 0.149597167
bestIteration = 80

1:	loss: 0.1495972	best: 0.1494285 (0)	total: 42s	remaining: 16m 48s

bestTest = 0.1494355126
bestIteration = 163

2:	loss: 0.1494355	best: 0.1494285 (0)	total: 1m 2s	remaining: 16m 24s

bestTest = 0.1491218322
bestIteration = 68

3:	loss: 0.1491218	best: 0.1491218 (3)	total: 1m 23s	remaining: 16m 3s

bestTest = 0.1490008908
bestIteration = 24

4:	loss: 0.1490009	best: 0.1490009 (4)	total: 1m 44s	remaining: 15m 40s

bestTest = 0.1506007902
bestIteration = 10

5:	loss: 0.1506008	best: 0.1490009 (4)	total: 2m 5s	remaining: 15m 20s

bestTest = 0.1488442623
bestIteration = 988

6:	loss: 0.1488443	best: 0.1488443 (6)	total: 2m 24s	remaining: 14m 50s

bestTest = 0.1497644535
bestIteration = 202

7:	loss: 0.1497645	best: 0.1488443 (6)	total: 2m 45s	remaining: 14m 30s

bestTest = 0.149748771
bestIteration = 78

8:	loss: 0.1497488	best: 

In [ ]:
c1 = CatBoostClassifier(loss_function='MultiClass')

In [ ]:
randomized_pet = c1.randomized_search(grid,
                  dataset1,
                  cv=3,
                  n_iter=50,
                  partition_random_seed=0,
                  calc_cv_statistics=True, 
                  search_by_train_test_split=True,
                  refit=True, 
                  shuffle=True, 
                  stratified=None, 
                  verbose=True)


bestTest = 0.2612626254
bestIteration = 133

0:	loss: 0.2612626	best: 0.2612626 (0)	total: 29.7s	remaining: 24m 14s

bestTest = 0.2324065922
bestIteration = 397

1:	loss: 0.2324066	best: 0.2324066 (1)	total: 58.8s	remaining: 23m 31s

bestTest = 0.2329034547
bestIteration = 827

2:	loss: 0.2329035	best: 0.2324066 (1)	total: 1m 27s	remaining: 22m 45s

bestTest = 0.2309456454
bestIteration = 498

3:	loss: 0.2309456	best: 0.2309456 (3)	total: 1m 55s	remaining: 22m 13s

bestTest = 0.2357698455
bestIteration = 206

4:	loss: 0.2357698	best: 0.2309456 (3)	total: 2m 24s	remaining: 21m 41s

bestTest = 0.238035394
bestIteration = 86

5:	loss: 0.2380354	best: 0.2309456 (3)	total: 2m 53s	remaining: 21m 14s

bestTest = 0.2384029957
bestIteration = 999

6:	loss: 0.2384030	best: 0.2309456 (3)	total: 3m 19s	remaining: 20m 26s

bestTest = 0.2316153947
bestIteration = 717

7:	loss: 0.2316154	best: 0.2309456 (3)	total: 3m 47s	remaining: 19m 53s

bestTest = 0.2339528846
bestIteration = 138

8:	loss: 0.233

In [ ]:
y_te_br = c.predict(X1)
y_te_br

array([[1.],
       [0.],
       [0.],
       ...,
       [1.],
       [2.],
       [1.]])

In [ ]:
y_te_br = pd.DataFrame(y_te_br)
y_te_br.head()

,0
0,1.0
1,0.0
2,0.0
3,0.0
4,0.0


In [ ]:
y_te_br.rename(columns={0:'breed_category'},inplace=True)
y_te_br.head()

,breed_category
0,1.0
1,0.0
2,0.0
3,0.0
4,0.0


In [ ]:
y_te_pet = c1.predict(X1)
y_te_pet

array([[2],
       [1],
       [2],
       ...,
       [2],
       [4],
       [2]])

In [ ]:
y_te_pet = pd.DataFrame(y_te_pet)
y_te_pet.head()

,0
0,2
1,1
2,2
3,2
4,2


In [ ]:
y_te_pet.rename(columns={0:'pet_category'},inplace=True)
y_te_pet.head()

,pet_category
0,2
1,1
2,2
3,2
4,2


In [ ]:
df_c = pd.concat([df1['pet_id'],y_te_br['breed_category'],y_te_pet['pet_category']],axis=1)  
df_c.head()

,pet_id,breed_category,pet_category
0,ANSL_75005,1.0,2
1,ANSL_76663,0.0,1
2,ANSL_58259,0.0,2
3,ANSL_67171,0.0,2
4,ANSL_72871,0.0,2


In [ ]:
df_c.to_csv('submission_30.csv', index=False)

In [ ]:
# sacing yhe model to be used at later stages #

import pickle

In [ ]:
!pip install catboost

     |████████████████████████████████| 65.9MB 44kB/s 


In [ ]:
from catboost import CatBoostClassifier

In [ ]:
pickle.dump(c, open('breed_cat.pkl', 'wb'))

In [ ]:
pickle.dump(c1, open('pet_cat.pkl', 'wb'))

In [ ]:
m = pickle.load(open('pet_cat.pkl','rb'))

In [ ]:
n = pickle.load(open('breed_cat.pkl','rb'))

In [ ]:
# Breed_category

y_t = n.predict(X1)
y_t

array([[1.],
       [0.],
       [0.],
       ...,
       [1.],
       [2.],
       [1.]])

In [ ]:
y_t = pd.DataFrame(y_t)
y_t.head()

,0
0,1.0
1,0.0
2,0.0
3,0.0
4,0.0


In [ ]:
# Breed Category #

y_t.rename(columns={0:'breed_category'},inplace=True)
y_t.head()

,breed_category
0,1.0
1,0.0
2,0.0
3,0.0
4,0.0


In [ ]:
# Pet Categry

y_t1 = m.predict(X1)
y_t1

array([[2],
       [1],
       [2],
       ...,
       [2],
       [4],
       [2]])

In [ ]:
y_t1

array([[2],
       [1],
       [2],
       ...,
       [2],
       [4],
       [2]])

In [ ]:
y_t1 = pd.DataFrame(y_t1)
y_t1.head()

,0
0,2
1,1
2,2
3,2
4,2


In [ ]:
# Pet Category #

y_t1.rename(columns={0:'pet_category'},inplace=True)
y_t1.head()

,pet_category
0,2
1,1
2,2
3,2
4,2


In [ ]:
# Concatenating the results of per and breed #

df_m1 = pd.concat([df1['pet_id'],y_t['breed_category'],y_t1['pet_category']],axis=1)  
df_m1.head()

,pet_id,breed_category,pet_category
0,ANSL_75005,1.0,2
1,ANSL_76663,0.0,1
2,ANSL_58259,0.0,2
3,ANSL_67171,0.0,2
4,ANSL_72871,0.0,2


In [ ]:
# Submission file #

df_m1.to_csv('submission_32.csv', index=False)  #score : 90.68

In [ ]:
# Creating a combined model #
# Breed category - catboost Model (breed_cat)
# Pet Category - Xgboost Model (pet_xgboost)

In [ ]:
# Breed Category # Predicted data is y_t
y_t.head()

,breed_category
0,1.0
1,0.0
2,0.0
3,0.0
4,0.0


In [ ]:
# Pet Category # Predicted Data is  y_pet_xg
y_pet_xg.head()

0    2
1    1
2    2
3    2
4    2
Name: pet_category, dtype: int64

In [ ]:
# Concatenating this data frames #


df_m2 = pd.concat([df1['pet_id'],y_t,y_pet_xg],axis=1)  
df_m2.head()


,pet_id,breed_category,pet_category
0,ANSL_75005,1.0,2
1,ANSL_76663,0.0,1
2,ANSL_58259,0.0,2
3,ANSL_67171,0.0,2
4,ANSL_72871,0.0,2


In [ ]:
df_m2.to_csv('submission_33.csv', index=False) #score : 90.71

In [ ]:
# New Model #

# Creating a combined model #
# Breed category - Random Forest Model (breed_rf), y_test_br
# Pet Category - Cat Model (pet_cat), y_t1

In [ ]:
# Concatenating this data frames #

df_m3 = pd.concat([df1['pet_id'],y_test_br,y_t1],axis=1)  
df_m3.head()

,pet_id,breed_category,pet_category
0,ANSL_75005,1.0,2
1,ANSL_76663,0.0,1
2,ANSL_58259,0.0,2
3,ANSL_67171,0.0,2
4,ANSL_72871,0.0,2


In [ ]:
df_m3.to_csv('submission_34.csv', index=False) #score :90.57